In [24]:
import pandas as pd
import json
import re

# Nom du fichier CSV à lire
df = pd.read_csv("C:\\Users\\ronan\\Desktop\\DataViz\\archive (4)\\job_descriptions.csv")

## Shape

In [26]:
print("\n------- Job Descritpion Dataframe --------\n")
df.info()


------- Job Descritpion Dataframe --------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Job Id            1615940 non-null  int64  
 1   Experience        1615940 non-null  object 
 2   Qualifications    1615940 non-null  object 
 3   Salary Range      1615940 non-null  object 
 4   location          1615940 non-null  object 
 5   Country           1615940 non-null  object 
 6   latitude          1615940 non-null  float64
 7   longitude         1615940 non-null  float64
 8   Work Type         1615940 non-null  object 
 9   Company Size      1615940 non-null  int64  
 10  Job Posting Date  1615940 non-null  object 
 11  Preference        1615940 non-null  object 
 12  Contact Person    1615940 non-null  object 
 13  Contact           1615940 non-null  object 
 14  Job Title         1615940 non-null  object 
 15  Role

## Cleaning

In [28]:
def visualisation_donnees_manquantes(dataframe):
    """
    Affiche les colonnes avec des données manquantes et leur pourcentage dans la DataFrame.
    Permet à l'utilisateur de choisir de supprimer les lignes contenant des données manquantes.
    
    Arguments :
    dataframe : pd.DataFrame : La DataFrame à analyser.
    
    Retourne :
    Une DataFrame mise à jour après la suppression des lignes choisies par l'utilisateur.
    """
    
    print("\n------- Resultats --------\n")    
    # Collecte des colonnes avec des données manquantes
    col_with_na = dataframe.columns[dataframe.isna().any()].tolist()
    
    # Calcul des statistiques
    stats = [
        (col, dataframe[col].isna().sum(), round(dataframe[col].isna().sum() / len(dataframe) * 100, 2))
        for col in col_with_na
    ]
    
    # Affichage des statistiques avec une fonction lambda
    afficher_stat = lambda stat: f"Colonne: {stat[0]}, Données manquantes: {stat[1]}, Pourcentage: {stat[2]}%"
    if stats:
        print("\n".join(map(afficher_stat, stats)))
    else:
        print("Aucune colonne avec des données manquantes.")
        return dataframe
    
    # Interaction utilisateur et suppression des lignes
    cols_to_drop = list(
        filter(
            lambda col: input(
                f"Souhaitez-vous supprimer les lignes avec des valeurs manquantes dans la colonne '{col}' ? (Y/N) : "
            ).strip().lower() == "y",
            col_with_na,
        )
    )
    
    if cols_to_drop:
        dataframe = dataframe.dropna(subset=cols_to_drop)
        print(f"Lignes contenant des données manquantes dans les colonnes {cols_to_drop} ont été supprimées.")
    else:
        print("Aucune suppression effectuée.")
    
    return dataframe

def extraction_colonne(colonne_initial, colonne):
    
    if isinstance(colonne_initial, str):
        try:
            colonne_sortie = (json.loads(colonne_initial))[colonne]
            return colonne_sortie.replace("&","and")
        except:
            return None
    else:
        return None


In [29]:
df = visualisation_donnees_manquantes(df)


------- Resultats --------

Colonne: Company Profile, Données manquantes: 5478, Pourcentage: 0.34%


Souhaitez-vous supprimer les lignes avec des valeurs manquantes dans la colonne 'Company Profile' ? (Y/N) :  y


Lignes contenant des données manquantes dans les colonnes ['Company Profile'] ont été supprimées.


## Extraire Secteur

In [31]:
df.loc[0, "Company Profile"]

'{"Sector":"Diversified","Industry":"Diversified Financials","City":"Sunny Isles Beach","State":"Florida","Zip":"33160","Website":"www.ielp.com","Ticker":"IEP","CEO":"David Willetts"}'

In [32]:
df["Company_Sector"] = df["Company Profile"].apply(lambda x: extraction_colonne(x, "Sector"))

## Convert Date in Date Time

In [34]:
# Convertir et extraction les dates

df['Job Posting Date']= pd.to_datetime(df['Job Posting Date'], errors='coerce')
df['Posting Month'] = df['Job Posting Date'].dt.month
df['Posting Year'] = df['Job Posting Date'].dt.year

In [35]:
df = visualisation_donnees_manquantes(df)


------- Resultats --------

Colonne: Company_Sector, Données manquantes: 1844, Pourcentage: 0.11%


Souhaitez-vous supprimer les lignes avec des valeurs manquantes dans la colonne 'Company_Sector' ? (Y/N) :  y


Lignes contenant des données manquantes dans les colonnes ['Company_Sector'] ont été supprimées.


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1608618 entries, 0 to 1615939
Data columns (total 26 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Job Id            1608618 non-null  int64         
 1   Experience        1608618 non-null  object        
 2   Qualifications    1608618 non-null  object        
 3   Salary Range      1608618 non-null  object        
 4   location          1608618 non-null  object        
 5   Country           1608618 non-null  object        
 6   latitude          1608618 non-null  float64       
 7   longitude         1608618 non-null  float64       
 8   Work Type         1608618 non-null  object        
 9   Company Size      1608618 non-null  int64         
 10  Job Posting Date  1608618 non-null  datetime64[ns]
 11  Preference        1608618 non-null  object        
 12  Contact Person    1608618 non-null  object        
 13  Contact           1608618 non-null  object     

## Period of time

In [38]:
# Trouver les dates minimum et maximum après conversion
start_date = df["Job Posting Date"].min()
end_date = df["Job Posting Date"].max()
period = end_date - start_date

print("Date de début :", start_date)
print("Date de fin :", end_date)
print("period:", period)


Date de début : 2021-09-15 00:00:00
Date de fin : 2023-09-15 00:00:00
period: 730 days 00:00:00


## Expend Salaries Informations (min /max)

In [40]:
#création salary min et max
df[['Salary min', 'Salary max']] = df['Salary Range'].str.split('-', expand = True)

#salaire min 
df['Salary min'] = df['Salary min'].str.replace('$', '').str.rstrip('K').astype(float)*1000

#salaire max
df['Salary max'] = df['Salary max'].str.replace('$', '').str.rstrip('K').astype(float)*1000

#salaire moyen


def calculate_average_salary(salary_range):

    if pd.isna(salary_range):

        return None

    try:
        salaries = salary_range.replace('$', '').replace('K', '000').split('-')

        return (float(salaries[0]) + float(salaries[1])) / 2 if len(salaries) == 2 else None

    except:

        return None



df['Average Salary'] = df['Salary Range'].apply(calculate_average_salary)

## Expend Experience informations 

In [42]:
df[['Experience min', 'Experience max']] = df['Experience'].str.split(' to ', expand=True)

# Convert string to integers
df['Experience min'] = df['Experience min'].str.replace(' Years', '').astype(int)
df['Experience max'] = df['Experience max'].str.replace(' Years', '').astype(int)

## Expend Size Company informations 

In [44]:
#Catégorisation de la taille de l’entreprise (Company Size Category)

def categorize_company_size(size):

    try:

        size = int(size)

        if size < 50:

            return 'Petite'

        elif size < 250:

            return 'Moyenne'

        else:

            return 'Grande'

    except:

        return 'Unknown'



df['Company Size Category'] = df['Company Size'].apply(categorize_company_size)

## Sectors and companies

In [46]:
Secteur_activite = df["Company_Sector"].nunique()

In [47]:
unique_companies_count = df["Company"].nunique()
print ('Le dataframe regroupe', unique_companies_count, 'entreprises appartenant à', Secteur_activite, "secteurs d'activités.")

Le dataframe regroupe 884 entreprises appartenant à 197 secteurs d'activités.


In [48]:
df = visualisation_donnees_manquantes(df)


------- Resultats --------

Aucune colonne avec des données manquantes.


## Attribution job_sector

In [50]:
dic_metier = {
       "Software Tester": "Technologie de l'Information",
       "Network Engineer": "Technologie de l'Information",
       'Database Administrator': "Technologie de l'Information",
       'Technical Writer': "Technologie de l'Information",
       'IT Manager': "Technologie de l'Information",
       "Network Administrator": "Technologie de l'Information",
       "Software Engineer": "Technologie de l'Information",
       "Data Analyst": "Technologie de l'Information",
       "Systems Analyst": "Technologie de l'Information",
       "Java Developer": "Technologie de l'Information",
       "Front-End Engineer": "Technologie de l'Information",
       "IT Manage": "Technologie de l'Information",
       "Data Engineer": "Technologie de l'Information",
       "Systems Engineer": "Technologie de l'Information",
       "Front-End Developer": "Technologie de l'Information",
       "UX/UI Designer": "Technologie de l'Information",
       "UI Developer": "Technologie de l'Information",
       "IT Administrator": "Technologie de l'Information",
       "QA Analyst": "Technologie de l'Information",
       "QA Engineer": "Technologie de l'Information",
       "Software Developer": "Technologie de l'Information",
       "Network Analyst": "Technologie de l'Information",
       "Network Security Specialist": "Technologie de l'Information",
       "Network Technician": "Technologie de l'Information",
       "Web Developer": "Technologie de l'Information",
       "Software Architect": "Technologie de l'Information",
       "IT Support Specialist": "Technologie de l'Information",
       "Back-End Developer": "Technologie de l'Information",
       "Systems Administrator": "Technologie de l'Information",
       "Data Scientist": "Technologie de l'Information",
       "Web Designer": "Technologie de l'Information",
       "Database Developer": "Technologie de l'Information",
       "Digital Marketing Specialist": "Marketing & Communication",
       "SEO Analyst": "Marketing & Communication",
       'Marketing Manager': "Marketing & Communication",
       "SEO Specialist": "Marketing & Communication",
       "Market Analyst": "Marketing & Communication",
       "Marketing Director": "Marketing & Communication",
       "Marketing Specialist": "Marketing & Communication",
       "Marketing Analyst": "Marketing & Communication",
       "Marketing Coordinator": "Marketing & Communication",
       "Brand Manager": "Marketing & Communication",
       "Content Writer": "Marketing & Communication",
       "Email Marketing Specialist": "Marketing & Communication",
       "Public Relations Specialist": "Marketing & Communication",
       "Social Media Manager": "Marketing & Communication",
       "Market Research Analyst": "Marketing & Communication",
       "SEM Specialist": "Marketing & Communication",
       "Social Media Coordinator": "Marketing & Communication",
       "Copywriter": "Marketing & Communication",
       "Occupational Therapist": "Santé & Soins",
       "Nurse Practitioner": "Santé & Soins",
       "Nurse Manager": "Santé & Soins",
       "Substance Abuse Counselor": "Santé & Soins",
       "Pharmaceutical Sales Representative": "Santé & Soins",
       "Physical Therapist": "Santé & Soins",
       "Dental Hygienist": "Santé & Soins",
       "Pediatrician": "Santé & Soins",
       "Family Nurse Practitioner": "Santé & Soins",
       "Physician Assistant": "Santé & Soins",
       "Registered Nurse": "Santé & Soins",
       "Speech Therapist": "Santé & Soins",
       "Veterinarian": "Santé & Soins",
       "HR Coordinator": "Gestion & Administration",
       'Data Entry Clerk': "Gestion & Administration",
       'Human Resources Manager': "Gestion & Administration",
       "Account Manager": "Gestion & Administration",
       "Account Director": "Gestion & Administration",
       "Account Executive": "Gestion & Administration",
       "Business Development Manager": "Gestion & Administration",
       "Procurement Specialist": "Gestion & Administration",
       "Project Coordinator": "Gestion & Administration",
       "Quality Assurance Analyst": "Gestion & Administration",
       "HR Manager": "Gestion & Administration",
       "HR Generalist": "Gestion & Administration",
       "Talent Acquisition Specialist": "Gestion & Administration",
       "Recruitment Consultant": "Gestion & Administration",
       "Mechanical Engineer": "Ingénierie",
       "Civil Engineer": "Ingénierie",
       "Electrical Engineer": "Ingénierie",
       "Structural Engineer": "Ingénierie",
       "Aerospace Engineer": "Ingénierie",
       "Chemical Engineer": "Ingénierie",
       "Chemical Analyst": "Ingénierie",
       "Environmental Engineer": "Ingénierie",
       "Electrical Designer": "Ingénierie",
       "Mechanical Designer": "Ingénierie",
       'Financial Analyst': 'Finance & Banque',
       "Investment Advisor": 'Finance & Banque',
       "Finance Manager": 'Finance & Banque',
       'Financial Planner': 'Finance & Banque',
       'Financial Advisor': 'Finance & Banque',
       'Investment Analyst': 'Finance & Banque',
       'Investment Banker': 'Finance & Banque',
       'Tax Consultant': 'Finance & Banque',
       'Accountant': 'Finance & Banque',
       'Financial Controller': 'Finance & Banque',
       "Teacher": "Éducation & Formation",
       "Research Analyst": "Éducation & Formation",
       "Art Teacher": "Éducation & Formation",
       "Research Scientist": "Éducation & Formation",
       "Graphic Designer": "Design & Créativité",
       "Architect": "Design & Créativité",
       "Product Manager": "Design & Créativité",
       "Product Designer": "Design & Créativité",
       "Interior Designer": "Design & Créativité",
       "UX Researcher": "Design & Créativité",
       "Architectural Designer": "Design & Créativité",
       'Litigation Attorney': 'Juridique',
       'Family Lawyer': 'Juridique',
       'Legal Advisor': 'Juridique',
       "Legal Assistant": 'Juridique',
       'Legal Counsel': 'Juridique',
       'Legal Secretary': 'Juridique',
       'Paralegal': 'Juridique',
       "Sales Representative": "Ventes & Support Client",
       "Sales Manager": "Ventes & Support Client",
       "Sales Associate": "Ventes & Support Client",
       "Customer Service Representative": "Ventes & Support Client",
       "Customer Support Specialist": "Ventes & Support Client",
       "Customer Service Manager": "Ventes & Support Client",
       "Customer Success Manager": "Ventes & Support Client",
       "Key Account Manager": "Ventes & Support Client",
       "Sales Consultant": "Ventes & Support Client",
       "Brand Ambassador": "Ventes & Support Client",
       "Procurement Manager": "Logistique & Supply Chain",
       "Supply Chain Manager": "Logistique & Supply Chain",
       "Inventory Analyst": "Logistique & Supply Chain",
       "Procurement Coordinator": "Logistique & Supply Chain",
       "Purchasing Agent": "Logistique & Supply Chain",
       "Supply Chain Analyst": "Logistique & Supply Chain",
       "Landscape Architect": "Immobilier & Construction",
       "Urban Planner": "Immobilier & Construction",
       "Landscape Designer": "Immobilier & Construction",
       "Construction Manager": "Immobilier & Construction",
       'Operations Manager': 'Divers (Gestion, Planification, Événementiel)',
       'Project Manager': 'Divers (Gestion, Planification, Événementiel)',
       'Executive Assistant': 'Divers (Gestion, Planification, Événementiel)',
       'Business Analyst': 'Divers (Gestion, Planification, Événementiel)',
       'Office Manager': 'Divers (Gestion, Planification, Événementiel)',
       'Administrative Assistant': 'Divers (Gestion, Planification, Événementiel)',
       'Process Engineer': 'Divers (Gestion, Planification, Événementiel)',
       "Event Planner": "Événementiel",
       "Event Coordinator": "Événementiel",
       "Event Manager": "Événementiel",
       "Wedding Planner": "Événementiel",
       "Environmental Consultant": "Autres Secteurs",
       "Art Director": "Autres Secteurs",
       "Psychologist": "Autres Secteurs",
       "Social Worker": "Autres Secteurs",
       "Personal Assistant": "Autres Secteurs"

}

In [51]:
df = visualisation_donnees_manquantes(df)


------- Resultats --------

Aucune colonne avec des données manquantes.


In [52]:
Europe = [
        'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 
        'Bulgaria', 'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Faroe Islands', 
        'Finland', 'France', 'Germany', 'Gibraltar', 'Greece', 'Hungary', 
        'Iceland', 'Ireland', 'Italy', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 
        'Luxembourg', 'Macedonia, FYR', 'Malta', 'Monaco', 'Montenegro', 'Netherlands', 
        'Norway', 'Poland', 'Portugal', 'Romania', 'San Marino', 'Serbia', 'Slovak Republic', 
        'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine', 'UK'
    ]
data_Europe = df[df['Country'].isin(Europe)].copy()
data_Europe

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Company Profile,Company_Sector,Posting Month,Posting Year,Salary min,Salary max,Average Salary,Experience min,Experience max,Company Size Category
5,116831420231957,4 to 12 Years,MCA,$59K-$93K,Brussels,Belgium,50.5039,4.4699,Full-Time,23196,...,"{""Sector"":""Infrastructure"",""Industry"":""Ports a...",Infrastructure,7,2023,59000.0,93000.0,76000.0,4,12,Grande
16,2823887164112482,5 to 14 Years,M.Tech,$60K-$83K,San Marino,San Marino,43.9424,12.4578,Part-Time,49100,...,"{""Sector"":""Industrial"",""Industry"":""Constructio...",Industrial,4,2022,60000.0,83000.0,71500.0,5,14,Grande
26,1504621983937621,5 to 15 Years,M.Tech,$59K-$88K,Budapest,Hungary,47.1625,19.5033,Contract,105370,...,"{""Sector"":""Food and Beverage"",""Industry"":""Food...",Food and Beverage,11,2022,59000.0,88000.0,73500.0,5,15,Grande
46,1373513885523551,3 to 9 Years,B.Tech,$56K-$109K,Madrid,Spain,40.4637,-3.7492,Temporary,91636,...,"{""Sector"":""Manufacturing"",""Industry"":""Metals"",...",Manufacturing,5,2023,56000.0,109000.0,82500.0,3,9,Grande
48,2283172037354283,3 to 8 Years,BA,$59K-$89K,Valletta,Malta,35.9375,14.3754,Temporary,99455,...,"{""Sector"":""Chemicals"",""Industry"":""Chemicals"",""...",Chemicals,1,2022,59000.0,89000.0,74000.0,3,8,Grande
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615919,974105432974512,4 to 12 Years,BA,$58K-$123K,Prague,Czech Republic,49.8175,15.4730,Part-Time,50310,...,"{""Sector"":""Beauty/Cosmetics"",""Industry"":""Consu...",Beauty/Cosmetics,9,2022,58000.0,123000.0,90500.0,4,12,Grande
1615928,1827059374762454,5 to 12 Years,B.Com,$58K-$115K,Sarajevo,Bosnia and Herzegovina,43.9159,17.6791,Contract,50966,...,"{""Sector"":""Retail"",""Industry"":""General Merchan...",Retail,12,2021,58000.0,115000.0,86500.0,5,12,Grande
1615930,690948368250854,1 to 14 Years,BCA,$65K-$125K,Madrid,Spain,40.4637,-3.7492,Intern,131847,...,"{""Sector"":""Food & Beverage"",""Industry"":""Food C...",Food and Beverage,7,2022,65000.0,125000.0,95000.0,1,14,Grande
1615932,1162938205421273,4 to 14 Years,B.Com,$64K-$130K,San Marino,San Marino,43.9424,12.4578,Full-Time,71583,...,"{""Sector"":""Energy"",""Industry"":""Energy - Utilit...",Energy,3,2023,64000.0,130000.0,97000.0,4,14,Grande


In [53]:
data_Europe["Job Title"]

5                   Software Tester
16                    Social Worker
26                 Sales Consultant
46                 Data Entry Clerk
48                   Tax Consultant
                     ...           
1615919              Office Manager
1615928              UX/UI Designer
1615930    Environmental Consultant
1615932                UI Developer
1615936                  IT Manager
Name: Job Title, Length: 327520, dtype: object

In [54]:
data_Europe = visualisation_donnees_manquantes(data_Europe)


------- Resultats --------

Aucune colonne avec des données manquantes.


In [55]:
type(data_Europe)

pandas.core.frame.DataFrame

In [56]:
data_Europe["Job_Sector"] = data_Europe["Job Title"].map(dic_metier)
data_Europe["Job_Sector"]

5                           Technologie de l'Information
16                                       Autres Secteurs
26                               Ventes & Support Client
46                              Gestion & Administration
48                                      Finance & Banque
                               ...                      
1615919    Divers (Gestion, Planification, Événementiel)
1615928                     Technologie de l'Information
1615930                                  Autres Secteurs
1615932                     Technologie de l'Information
1615936                     Technologie de l'Information
Name: Job_Sector, Length: 327520, dtype: object

In [57]:
data_Europe[["Company","Company_Sector", "Job_Sector"]]

,Company,Company_Sector,Job_Sector
5,Adani Ports and Special Economic Zone,Infrastructure,Technologie de l'Information
16,Deere,Industrial,Autres Secteurs
26,Associated British Foods,Food and Beverage,Ventes & Support Client
46,Arconic,Manufacturing,Gestion & Administration
48,Dow,Chemicals,Finance & Banque
...,...,...,...
1615919,"Revlon, Inc.",Beauty/Cosmetics,"Divers (Gestion, Planification, Événementiel)"
1615928,Walmart,Retail,Technologie de l'Information
1615930,J.M. Smucker,Food and Beverage,Autres Secteurs
1615932,Drax Group,Energy,Technologie de l'Information


In [58]:
data_Europe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 327520 entries, 5 to 1615936
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Job Id                 327520 non-null  int64         
 1   Experience             327520 non-null  object        
 2   Qualifications         327520 non-null  object        
 3   Salary Range           327520 non-null  object        
 4   location               327520 non-null  object        
 5   Country                327520 non-null  object        
 6   latitude               327520 non-null  float64       
 7   longitude              327520 non-null  float64       
 8   Work Type              327520 non-null  object        
 9   Company Size           327520 non-null  int64         
 10  Job Posting Date       327520 non-null  datetime64[ns]
 11  Preference             327520 non-null  object        
 12  Contact Person         327520 non-null  object  

In [59]:
# Fonction de nettoyage des compétences
def clean_skills(skills):
    if pd.isna(skills):  # Vérifier les valeurs manquantes
        return []
    
    # Supprimer les parenthèses contenant des compétences séparées par des virgules
    skills = re.sub(r"\(([^)]+)\)", lambda x: ', '.join(x.group(1).split(',')), skills)
    
    # Supprimer les parenthèses restantes et autres caractères inutiles
    skills = re.sub(r"[()]", "", skills)
    skills = skills.replace("e.g.,", "").strip()
    
    # Séparer les compétences par espace suivi d'une majuscule ou virgule
    split_skills = re.split(r",\s*|\s+(?=[A-Z])", skills)
    
    # Supprimer les doublons, nettoyer les espaces et entourer chaque compétence de guillemets simples
    return [f'{skill.strip()}' for skill in dict.fromkeys(split_skills) if skill.strip()]

# Appliquer le nettoyage à la colonne 'skills'
data_Europe["cleaned_skills"] = data_Europe["skills"].apply(clean_skills)

# Afficher un aperçu des résultats
print(data_Europe[["skills", "cleaned_skills"]])


                                                    skills  \
5        Quality assurance processes Testing methodolog...   
16       School social work principles Student assessme...   
26       B2B sales Business-to-business sales strategie...   
46       Records management Data entry and retrieval At...   
48       International tax law knowledge Transfer prici...   
...                                                    ...   
1615919  Facilities management Property maintenance Spa...   
1615928  UI design principles and best practices Graphi...   
1615930  Sustainability consulting Sustainability asses...   
1615932  User experience (UX) design User interface (UI...   
1615936  Strategic IT planning Leadership and managemen...   

                                            cleaned_skills  
5        [Quality assurance processes, Testing methodol...  
16       [School social work principles, Student assess...  
26       [B2B sales, Business-to-business sales strateg...  
46       [R

In [60]:
data_Europe[["skills", "cleaned_skills"]]

,skills,cleaned_skills
5,Quality assurance processes Testing methodolog...,"[Quality assurance processes, Testing methodol..."
16,School social work principles Student assessme...,"[School social work principles, Student assess..."
26,B2B sales Business-to-business sales strategie...,"[B2B sales, Business-to-business sales strateg..."
46,Records management Data entry and retrieval At...,"[Records management, Data entry and retrieval,..."
48,International tax law knowledge Transfer prici...,"[International tax law knowledge, Transfer pri..."
...,...,...
1615919,Facilities management Property maintenance Spa...,"[Facilities management, Property maintenance, ..."
1615928,UI design principles and best practices Graphi...,"[UI design principles and best practices, Grap..."
1615930,Sustainability consulting Sustainability asses...,"[Sustainability consulting, Sustainability ass..."
1615932,User experience (UX) design User interface (UI...,"[User experience, UX design, User interface, U..."


In [93]:
data_export_info = data_Europe[["Job Title", "Country", "location", "latitude", "longitude", "Work Type", "Preference", "Qualifications", "skills", "Responsibilities", "Job_Sector","Salary min", "Average Salary" ,"Salary max", "Experience min","Job Posting Date","Posting Month","Posting Year","Company_Sector","Company"]]

## Export to CSV

In [97]:
# Sauvegarder le fichier nettoyé
data_export_info = data_Europe[["Job Title", "Country", "location", "latitude", "longitude", "Work Type", "Preference",
                                "Qualifications", "cleaned_skills", "Responsibilities", "Job_Sector","Salary min","Salary max", "Average Salary", 
                                "Experience min","Job Posting Date","Posting Month","Posting Year","Company_Sector","Company"]].rename(columns={"cleaned_skills": "skills"})

data_export_info.to_csv("data_export.csv", index=False)

# Sélectionner les colonnes nécessaires et renommer 'cleaned_skills' en 'skills'
data_export_skills_by_job = data_Europe[["Job Title", "cleaned_skills"]].rename(columns={"cleaned_skills": "skills"})

# Exporter le DataFrame en fichier CSV
data_export_skills_by_job.to_csv("skills_by_job.csv", index=False)

print("Export terminé : 'skills_by_job.csv'")

Export terminé : 'skills_by_job.csv'
